In [1]:
import vk_api
from vk_api.exceptions import ApiError
import pandas as pd
from math import ceil
from datetime import datetime
from tqdm import tqdm

In [23]:
ACCESS_TOKEN = 'ACCESS_TOKEN'
GROUP_ID = -60463565 #the '-' sign indicate where it is a group, not user
MAX_POST_COUNT = 10000

Documentation for posts and comments data objects:
- https://dev.vk.com/ru/reference/objects/post
- https://dev.vk.com/ru/reference/objects/comment


In [3]:
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)

vk = vk_session.get_api()

# Get posts list

In [20]:
response = vk.wall.get(owner_id=GROUP_ID, count=100, extended=1)
response

{'count': 72425,
 'items': [{'inner_type': 'wall_wallpost',
   'comments': {'can_post': 1, 'count': 2, 'groups_can_post': True},
   'marked_as_ads': 0,
   'hash': 'EEJ09MzVri2J5thC1A',
   'type': 'post',
   'attachments': [],
   'date': 1703621641,
   'from_id': -60463565,
   'id': 2651040,
   'likes': {'can_like': 1,
    'count': 7,
    'user_likes': 0,
    'can_publish': 1,
    'repost_disabled': False},
   'owner_id': -60463565,
   'post_source': {'type': 'vk'},
   'post_type': 'post',
   'reposts': {'count': 26, 'user_reposted': 0},
   'signer_id': 410882666,
   'text': 'Ежедневная разовая подработка для мужчин \nЗадача: приехать к нам в студию и сфотографироваться ( для обучения нейронной сети по обработке портретов) \nКритерии: Без синяков на лице, без тату, пирсинг прячем \nЗанятость: 20-30мин \nоплата: 700р сразу на месте ( наличными\\перевод) \nДля всех желающих, кто ещё не принимал участия в подобных мероприятиях. \nТакже, у нас есть бонусная система по приводу друзей. \nСтуд

In [24]:
response['items'][0]['comments']['count']

2

In [26]:
posts_ids = []
print(f'{datetime.now()} Start requesting data')
for i in tqdm(range(ceil(MAX_POST_COUNT / 100))):
    data = vk.wall.get(owner_id=GROUP_ID, count=100, extended=1, offset=i*100)
    for post in data['items']:
        if post['comments']['count'] > 0:
            posts_ids.append(post['id'])
print(f'{datetime.now()} End requesting data')
print(f'There are {len(posts_ids)} posts with at least one comment')

2023-12-27 01:46:04.530768 Start requesting data


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:56<00:00,  1.17s/it]

2023-12-27 01:48:01.190097 End requesting data
There are 9027 posts with at least one comment


In [49]:
posts_ids[21]

2650461

# Get likes and comments

In [51]:
res = vk.likes.getList(type='comment', owner_id=-60463565, item_id=2650461)
res

{'count': 0, 'items': []}

In [52]:
res = vk.wall.getComments(owner_id=-60463565, post_id=2650461, need_likes=1, count=100, extended=1, thread_items_count=10)
res

{'count': 38,
 'items': [{'id': 2650462,
   'from_id': 492961741,
   'date': 1703486346,
   'text': 'Вы у комментаторов решили спросить насколько уверены в своих отношениях?',
   'post_id': 2650461,
   'owner_id': -60463565,
   'parents_stack': [],
   'likes': {'can_like': 1,
    'count': 1,
    'user_likes': 0,
    'can_like_by_group': 0,
    'group_liked': False},
   'thread': {'count': 1,
    'items': [{'id': 2650653,
      'from_id': 734110767,
      'date': 1703504647,
      'text': '[id492961741|Оксана], не очень уверены.',
      'post_id': 2650461,
      'owner_id': -60463565,
      'parents_stack': [2650462],
      'attachments': [{'type': 'photo',
        'photo': {'album_id': -8,
         'date': 1703504647,
         'id': 457769038,
         'owner_id': -60463565,
         'access_key': '6e2e519f974a3f2c77',
         'sizes': [{'height': 72,
           'type': 's',
           'width': 75,
           'url': 'https://sun9-10.userapi.com/impf/dcTpWuC-lhgn_5Nm5P2cOLPV4lErTlpow3P

In [64]:
def parse_reply(reply):
    data=[]
    data.append([reply['id'] if 'id' in reply else None, 
                reply['post_id'] if 'post_id' in reply else None,
                0, #reply_to_comment
                reply['from_id'] if 'from_id' in reply else None,
                'reply', #type
                reply['text'] if 'text' in reply else None]
            )
    likes = vk.likes.getList(type='comment', owner_id=GROUP_ID, item_id=reply['id'])
    for item in likes['items']:
        data.append([reply['id']+item if 'id' in reply else item, #artificial identifier for like
                    reply['post_id'] if 'post_id' in reply else None,
                    reply['id'] if 'id' in reply else None,
                    item,
                    'like',
                    '']
                )
    return data

In [65]:
def parse_thread(thread):
    data=[]
    for item in thread['items']:
        data.append([item['id'] if 'id' in item else None, 
                    item['post_id'] if 'post_id' in item else None,
                    item['reply_to_comment'] if 'reply_to_comment' in item else None,
                    item['from_id'] if 'from_id' in item else None,
                    'reply', #type
                    item['text'] if 'text' in item else None]
                )
        likes = vk.likes.getList(type='comment', owner_id=GROUP_ID, item_id=item['id'])
        for like in likes['items']:
            data.append([item['id']+like if 'id' in item else like, #artificial identifier for like
                        item['post_id'] if 'post_id' in item else None,
                        item['id'] if 'id' in item else None,
                        like,
                        'like',
                        '']
                    )
    return data

In [67]:
user_interaction = []
print(f'{datetime.now()} Start requesting data')
for post in tqdm(posts_ids[:1000]):
    replies = vk.wall.getComments(owner_id=GROUP_ID, post_id=post, need_likes=1, count=100, extended=1, thread_items_count=10)
    for reply in replies['items']:
        reply_data = parse_reply(reply)
        user_interaction.extend(reply_data)
        if 'thread' in reply:
            thread_data = parse_thread(reply['thread'])
            user_interaction.extend(thread_data)
                
print(f'{datetime.now()} End requesting data')
print(f'There are {len(user_interaction)} data about different users interactions')

2023-12-27 10:24:58.856985 Start requesting data


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:18:15<00:00, 11.90s/it]

2023-12-27 13:43:14.318414 End requesting data
There are 87483 data about different users interactions


In [71]:
user_interaction[150]

[2651017,
 2650879,
 2650974,
 751541453,
 'reply',
 '[id768849819|Мефедрона], могу посоветовать БелАЗ😉😁']

In [74]:
interactions = pd.DataFrame(user_interaction, 
                     columns=['id','post_id', 'reply_to_comment', 
                              'from_id', 'type', 'text']
                    ).drop_duplicates()

interactions.head(10)

,id,post_id,reply_to_comment,from_id,type,text
0,2651063,2651040.0,0,34598232,reply,Отсосете мне и всех кого приведу
1,813962309,2651040.0,2651063,811311246,like,
2,495612804,2651040.0,2651063,492961741,like,
3,332599611,2651040.0,2651063,329948548,like,
4,794111148,2651040.0,2651063,791460085,like,
5,2651073,2651040.0,0,465882971,reply,А потом эта нейройная сеть будет разводить дру...
6,813962319,2651040.0,2651073,811311246,like,
7,673637608,2651040.0,2651073,670986535,like,
8,794111158,2651040.0,2651073,791460085,like,
9,2651090,2651040.0,0,575085622,reply,


In [75]:
interactions.to_csv('interactions.csv', index=False)